In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1AyULTkscFRSlZIJ143yXe7qg8fSNPBp-'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('util.py')

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1IugJe_XgxWtVkim24TlGz36YiUwGX13D'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('alexnet.py')

In [0]:
import sys
import os.path
import time
import alexnet
import tensorflow as tf
import util as tu
import numpy as np
import threading

In [0]:
def train(threads_numbers,epochs,batch_size,learning_rate,dropout,lmbda,resume,imagenet_path,display_step,test_step,ckpt_path,summary_path):
  """Procedure to train the model on ImageNet ILSVRC 2012 training set
        Args:
            resume: boolean variable,true if want to resume the training ,false to train from scratch
            imagenet_path: path to ILSVRC12 ImageNet folder containing train images,
                              validation images, annotations and metadata file
            display_step:number representing how often printing the current training accuracy
            test_step: number representing how often make a test and print the validation accuracy
            ckpt_path: path where to save model's tensorflow checkpoint(or from where resume)
            summary_path: path where to save logs for Tensorboard
  """
  train_img_path = os.path.join(imagenet_path,'ILSVRC2012_img_train')
  ts_size = tu.imagenet_size(train_img_path)
  num_batches = int(float(ts_size)/batch_size)
  
  wnids_labels,_ = tu.load_imagenet_meta(os.path,join(imagenet_path,'data/meta.mat'))
  with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32,[None,224,224,3])
    y = tf.placeholder(tf.float32,[None,1000])
    lr = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)
    
  #queue of examples being filled on the cpu
  
  with tf.device('/cpu:0'):
    q = tf.FIFOQueue(batch_size*3,[tf.float32,tf.float32],shapes = [[224,224,3],[1000]])
    enqueue_op = q.enqueue_many([x,y])
    x_b,y_b = q.dequeue_many(batch_size)
    
  pred,_ =alexnet.classifier(x_b,keep_prob)
  with tf.device('/gpu:0'):
    # cross-entorpy and weight decay
    with tf.name_scope('cross_entorpy'):
      cross_entropy =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred,labels = y_b,name = 'cross-entropy'))
  
    with tf.name_scope('l2_loss'):
      l2_loss = tf.reduce_sum(lmbda*tf.stack([tf.nn.l2_loss(v) for v in tf.get_collection('weights')]))
      tf.summary.scalar('l2_loss',l2_loss)
      
    with tf.name_scope('loss'):
      loss = cross_entrop+l2_loss
      tf.summary.scalar('loss',loss)
    
    #accuracy
    with tf.name_scope('accuracy'):
      correct = tf.equal(tf.argmax(pred,1),tf.argmax(y_b,1))
      accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
      tf.summary.scalar('accuracy',accuracy)
      
    global_step = tf.Variable(0,trainable = False)
    epoch = tf.div(global_step,num_batches)
    
    #momentum optimizer
    with tf.name_scope('optimizer'):
      optimizer = tf.train.AdamOptimizer(learning_rate = lr).minize(loss,global_step = global_step)
      
    #merge summaries to write them to file
    merged = tf.summary.merge_all()
    
    #checkpoint saver
    saver = tf.train.Saver()
    
    
    coord = tf.train.Coordinator()
    
    #init = tf.initialize_all_variables()
    init = tf.global_variables_initializer()
    
  with tf.Session(config = tf.ConfigProto(allow_soft_placement = True,log_device_placement =True)) as sess:
    if resume:
      saver.restore(sess,os.path.join(ckpt_path,'alexnet-cnn.ckpt'))
    else:
      sess.run(init)
      
    #enqueing batches procedure
    def enqueue_batches():
      while not coord.should_stop():
        im,l = tu.read_batch(batch_size,train_img_path,wnid_labels)
        sess.run(enqueue_op,feed_dict={x:im,y:1})
        
    #creating and starting parallel threads to fill the queue
    num_threads = threads_numbers
    
    for i in range(num_threads):
      t = threading.Thread(target = enqueue_batches)
      t.setDaemon(True)
      t.start()
      
    #operation to write logs for tensorboard visualization
    train_writer = tf.summary.FileWriter(os.path.join(summary_path,'train'),sess.graph)
    
    start_time = time.time()
    
    for e in range(sess.run(epoch),epochs):
      
      for i in range(num_batches):
        _,step = sess.run([optimizer,global_step],feed_dict = {lr: learning_rate,keep_prob: dropout})
        #train_writer.add_summary(summary,step)
        
        
        #decaying learning rate
        if step == 170000 or step == 350000:
          learning_rate /= 10
          
        # display current training informations
        if step% display_step == 0:
          temp_time = time.time()
          c,a = sess.run([loss,accuracy],feed_dict = {lr:learning_rate,keep_prob:1.0})
          print("time: ",temp_time-start_time,'Epoch: {:03d} Step/Batch: {:09d} --- Loss: {:.7f} Training accuracy: {:.4f}'.format(e,step,c,a))
          
        # make test and evaluate validation accuracy
        if step% test_step == 0:
          val_im,val_cls = tu.read_validation_batch(batch_size,os.path.join(imagenet_path,'ILSVRC2012_img_val'),os.path.join(imagenet_path,'data/ILSVRC2012_validation_ground_truth.txt'))
          v_a = sess.run(accuracy,feed_dict = {x_b:val_im,y_b:val_cls,lr:learning_rate,keep_prob:1.0})
          #intermediate time
          int_time = time.time()
          print('Elapsed time: {}'.format(tu.format(tu.format_time(int_time- start_time))))
          print('Validation accuracy: {:.04f}'.format(v_a))
          
          #save weights to file
          
          save_path = saver.save(sess,os.path.join(ckpt_path,'alexnet-cnn.ckpt'))
          print('Variables saved in file: %s'% save_path)
          
    end_time = time.time()
    print('Elapsed time:{}'.format(tu.format_time(end_time-start_time)))
    save_path = saver.save(sess,os.path.join(ckpt_path,'alexnet-cnn.ckpt'))
    print('Variables saved in file:%s' % save_path)
    
    coord.request_stop()
    
if __name__ == '__main__'    :
  Threads_numbers = 4
  DROPOUT = 0.5
  LAMBDA = 5e-05 #for weight decay
  LEARNING_RATE = 1e-03
  EPOCHS = 90
  BATCH_SIZE = 128
  CKPT_PATH = 'ckpt-alexnet'
  
  if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
  
  SUMMARY = 'summary'
  
  if not os.path.exists(SUMMARY):
    os.makedirs(SUMMARY)
    
  IMAGENET_PATH = 'F64E50644E502023/ILSVRC2012'
  DISPLAY_STEP = 10
  TEST_STEP = 500
  if len(sys.argv) == 1:
    resume = False
    
  elif sys.argv[1] == '-resume':
    resume = True
    
  train(Threads_numbers,EPOCHS,BATCH_SIZE,LEARNING_RATE,DROPOUT,LAMBDA,resume,IMAGENET_PATH,DISPLAY_STEP,TEST_STEP,CKPT_PATH,SUMMARY)